In [1]:
import numpy as np
from scipy.io.wavfile import write
from scipy.special import expit

import matplotlib.pyplot as plt

In [2]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio

In [3]:
import ipywidgets as widgets

In [4]:
def brown_noise(length):
    noise = np.cumsum(np.random.randn(length))
    return noise / np.max(np.abs(noise))

In [9]:
# Design a Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Apply the bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return lfilter(b, a, data)

# Filter the noise
def white_noise_filtered(num_samples, lowcut = 20.0, highcut = 250.0):
    white_noise = np.random.normal(0, 1, num_samples)
    filtered_noise = bandpass_filter(white_noise, lowcut, highcut, fs)
    return filtered_noise

In [10]:
def generate_sound(signal_type, duration, attack_duration, release_duration, slope_attack, slope_release, frequency=440, sr = 44100):
    num_samples = int(sr * duration)
    time = np.linspace(0, duration, num_samples)
    if signal_type == 'white':
        signal = np.random.normal(0, 1, num_samples)
    elif signal_type == 'sin':
        signal = np.sin(2 * np.pi * frequency * time)
    elif signal_type == 'brown':
        signal = brown_noise(num_samples)
    elif signal_type == 'white_filtr':
        signal = brown_noise(num_samples)
    
    # Create modified envelope with slope parameter
    attack = expit((time - attack_duration / 2) * slope_attack / attack_duration)
    release = expit((duration - time - release_duration / 2) * slope_release / release_duration)
    
    envelope = attack * release
    
    modulated_noise = signal * envelope
    modulated_noise = modulated_noise / np.max(np.abs(modulated_noise))

    return modulated_noise, envelope, time, sr

In [7]:
def update_envelope_prep(signal_selector, freq):
    def update_envelope(attack_duration, release_duration, attack_slope, release_slope):
        duration = 0.5
        signal_type = signal_selector
        frequency = freq
        modulated_noise, envelope, time, sr = generate_sound(signal_type, duration, attack_duration, release_duration, attack_slope, release_slope, frequency=frequency)
        
        fig, ax = plt.subplots(2,1, figsize=(12, 6))
        ax[0].plot(time, envelope)
        librosa.display.waveshow(modulated_noise, sr=sr, alpha=0.8, ax=ax[1])
        plt.title("Waveform")
        plt.xlabel("Time [s]")
        plt.ylabel("Amplitude")
        plt.grid()
        plt.show()
        
        display(Audio(modulated_noise, rate=sr))
    return update_envelope

In [11]:
attack_duration_slider = widgets.FloatSlider(value=0.5, min=0.05, max=1.0, step=0.05, description="Attack Dur:")
release_duration_slider = widgets.FloatSlider(value=0.5, min=0.05, max=1.0, step=0.05, description="Release Dur:")
attack_slope_slider = widgets.FloatSlider(value=10, min=1, max=20, step=1, description="Attack Slope:")
release_slope_slider = widgets.FloatSlider(value=10, min=1, max=20, step=1, description="Release Slope:")

# Link sliders to update function
signal_selector = 'white_filtr'
freq = 120

widgets.interactive(update_envelope_prep(signal_selector, freq), 
                    attack_duration=attack_duration_slider, 
                    release_duration=release_duration_slider, 
                    attack_slope=attack_slope_slider, 
                    release_slope=release_slope_slider)

# Display sliders
output = widgets.interactive_output(
    update_envelope_prep(signal_selector, freq),
    {
        "attack_duration": attack_duration_slider,
        "release_duration": release_duration_slider,
        "attack_slope": attack_slope_slider,
        "release_slope": release_slope_slider,
    },
)

# Display sliders and output together
ui = widgets.VBox([attack_duration_slider, release_duration_slider, attack_slope_slider, release_slope_slider])
display(ui, output)

Output()